## Librerías

In [1]:
install.packages("hdm")


Installing package into 'C:/Users/ammt2/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'hdm' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ammt2\AppData\Local\Temp\RtmpMn6333\downloaded_packages


In [2]:
library(glmnet)

Warning message:
"package 'glmnet' was built under R version 4.3.3"
Loading required package: Matrix

Loaded glmnet 4.1-8



In [3]:
library(hdm)


Warning message:
"package 'hdm' was built under R version 4.3.3"


In [4]:
install.packages("xtable")
install.packages("hdm")
install.packages("glmnet")
install.packages("ggplot2")
install.packages("tidyr")


Installing package into 'C:/Users/ammt2/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'xtable' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ammt2\AppData\Local\Temp\RtmpMn6333\downloaded_packages


Warning message:
"package 'hdm' is in use and will not be installed"
Warning message:
"package 'glmnet' is in use and will not be installed"
Installing package into 'C:/Users/ammt2/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'ggplot2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ammt2\AppData\Local\Temp\RtmpMn6333\downloaded_packages


Installing package into 'C:/Users/ammt2/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'tidyr' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'tidyr'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Users\ammt2\AppData\Local\R\win-library\4.3\00LOCK\tidyr\libs\x64\tidyr.dll  a C:\Users\ammt2\AppData\Local\R\win-library\4.3\tidyr\libs\x64\tidyr.dll: Permission denied"
Warning message:
"restored 'tidyr'"



The downloaded binary packages are in
	C:\Users\ammt2\AppData\Local\Temp\RtmpMn6333\downloaded_packages


In [5]:
url <- "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"
data <- read.csv(url)



## 1. Manejo de los datos

In [6]:
## Generación del modelo extra flexible
x_extra_flex <- paste("sex + (exp1 + exp2 + exp3 + exp4 + shs + hsg + scl + clg + C(occ2) + C(ind2) + mw + so + we)^2")
formula_extra_flex <- as.formula(paste("lwage", "~", x_extra_flex, "-1"))

model_x_extra_flex <- model.matrix(formula_extra_flex, data)

x<- as.data.frame(model_x_extra_flex)



In [7]:
x <- x[, colSums(x != 0) > 0]


In [8]:
y<-data$lwage

In [9]:
## Sepración en data de testeo y entrenamiento
data <- data.frame(y = y, x)

n <- nrow(data)

set.seed(123)

train_indices <- sample(1:n, size = 0.9 * n)

train_data <- data[train_indices, ]
test_data <- data[-train_indices, ]

x_train <- train_data[ , -1]  
y_train <- train_data$y

x_test <- test_data[ , -1] 
y_test <- test_data$y

data_test <- x_test
data_train <- x_train



In [10]:
model_x_extra_flex_train <- as.matrix(x_train)
model_x_extra_flex_test <- as.matrix(x_test)

p_flex <- dim(model_x_extra_flex_train)[2]




## 2.

In [11]:
generate_logarithmic_grid <- function(ln_lower_bound, ln_upper_bound, ln_spacing) {

  lower_bound <- exp(ln_lower_bound)
  upper_bound <- exp(ln_upper_bound)
  spacing <- exp(ln_spacing)
  
  grid <- c()
  value <- lower_bound
  while (value <= upper_bound) {
    grid <- c(grid, value)
    value <- value * spacing
  }
  
  return(grid)
}

ln_lower_bound <- log(1)
ln_upper_bound <- log(100)
ln_spacing <- log(2)  

log_grid <- generate_logarithmic_grid(ln_lower_bound, ln_upper_bound, ln_spacing)
print(log_grid)

[1]  1  2  4  8 16 32 64


In [12]:
generate_k_folds <- function(array, k) {
  n_rows <- nrow(array)
  indices <- sample(1:n_rows)
  fold_sizes <- rep(floor(n_rows / k), k)
  fold_sizes[1:(n_rows %% k)] <- fold_sizes[1:(n_rows %% k)] + 1
  
  folds <- vector("list", k)
  current <- 1
  for (i in 1:k) {
    fold_indices <- indices[current:(current + fold_sizes[i] - 1)]
    fold <- rep(FALSE, n_rows)
    fold[fold_indices] <- TRUE
    folds[[i]] <- fold
    current <- current + fold_sizes[i]
  }
  
return(folds)
}

In [13]:
# Caso de ejemplo
set.seed(123)
array <- matrix(runif(1000), nrow = 100, ncol = 10)  
k <- 5  
folds <- generate_k_folds(array, k)
#Generación 
for (i in 1:k) {
  cat("Pliegue", i, ":", folds[[i]], "\n")
}

Pliegue 1 : FALSE TRUE FALSE FALSE FALSE TRUE FALSE TRUE FALSE FALSE FALSE FALSE TRUE TRUE TRUE FALSE TRUE FALSE TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE TRUE TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE TRUE TRUE FALSE FALSE TRUE FALSE TRUE TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE TRUE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE 
Pliegue 2 : FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE TRUE FALSE TRUE TRUE FALSE TRUE FALSE FALSE FALSE FALSE FALSE TRUE FALSE TRUE FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE TRUE FALSE TRUE FALSE FALSE FALSE FALSE FALSE TRUE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE TRUE FALSE FALSE FALSE FALSE FALSE FA

In [14]:
find_optimal_lambda <- function(Y, X, lambda_bounds, k) {
  log_spacing <- 0.5
  lambdas <- generate_logarithmic_grid(lambda_bounds[1], lambda_bounds[2], log_spacing)
  folds <- generate_k_folds(X, k)
  
  all_mse <- numeric(length(lambdas))
  for (i in seq_along(lambdas)) {
    lmbda <- lambdas[i]
    mse_folds <- numeric(k)
    for (j in 1:k) {
      train_X <- X[!folds[[j]], ]
      test_X <- X[folds[[j]], ]
      train_Y <- Y[!folds[[j]]]
      test_Y <- Y[folds[[j]]]
      
      model <- glmnet(train_X, train_Y, alpha = 1, lambda = lmbda)
      predictions <- predict(model, test_X)
      mse_folds[j] <- mean((test_Y - predictions)^2)
    }
    all_mse[i] <- mean(mse_folds)
  }
  
  optimal_index <- which.min(all_mse)
  optimal_lambda <- lambdas[optimal_index]
  
  optimal_model <- glmnet(X, Y, alpha = 1, lambda = optimal_lambda)
  optimal_coef <- as.vector(coef(optimal_model))
  
  return(list(
    optimal_lambda = optimal_lambda,
    optimal_coef = optimal_coef,
    all_lambdas = lambdas,
    all_mse = all_mse
  ))
}

In [15]:
set.seed(123)
Y <- runif(100)  
X <- matrix(runif(1000), nrow = 100, ncol = 10) 
lambda_bounds <- c(0.01, 10)
k <- 5

optimal_model <- find_optimal_lambda(Y, X, lambda_bounds, k)
print(optimal_model)

predict_with_optimal_model <- function(optimal_model, X) {
  optimal_coef <- optimal_model$optimal_coef
  return(X %*% optimal_coef[-1] + optimal_coef[1])
}

predictions <- predict_with_optimal_model(optimal_model, X)
print(predictions)

$optimal_lambda
[1] 1.01005

$optimal_coef
 [1] 0.498559 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
 [9] 0.000000 0.000000 0.000000

$all_lambdas
 [1]     1.010050     1.665291     2.745601     4.526731     7.463317
 [6]    12.304930    20.287400    33.448268    55.146871    90.921819
[11]   149.904736   247.151127   407.483320   671.826418  1107.654505
[16]  1826.213543  3010.917113  4964.163088  8184.521275 13493.994316

$all_mse
 [1] 0.08207784 0.08207784 0.08207784 0.08207784 0.08207784 0.08207784
 [7] 0.08207784 0.08207784 0.08207784 0.08207784 0.08207784 0.08207784
[13] 0.08207784 0.08207784 0.08207784 0.08207784 0.08207784 0.08207784
[19] 0.08207784 0.08207784

           [,1]
  [1,] 0.498559
  [2,] 0.498559
  [3,] 0.498559
  [4,] 0.498559
  [5,] 0.498559
  [6,] 0.498559
  [7,] 0.498559
  [8,] 0.498559
  [9,] 0.498559
 [10,] 0.498559
 [11,] 0.498559
 [12,] 0.498559
 [13,] 0.498559
 [14,] 0.498559
 [15,] 0.498559
 [16,] 0.498559
 [17,] 0.498559
 [18,] 0.498559

## 3. 

### OLS

In [16]:
fit_extra_flex <- lm(y_train ~ ., data = x_train)


In [17]:
fit_extra_flex


Call:
lm(formula = y_train ~ ., data = x_train)

Coefficients:
        (Intercept)                  sex                 exp1  
          1.7964399           -0.0549805            0.3274346  
               exp2                 exp3                 exp4  
         -2.5489064            1.2568060           -0.7420854  
                shs                  hsg                  scl  
         -2.6833425           -1.5511535           -1.0512200  
                clg             C.occ2.1             C.occ2.2  
         -0.2622654            2.2122013            0.9771225  
           C.occ2.3             C.occ2.4             C.occ2.5  
          2.2909053            1.2375986            0.7618742  
           C.occ2.6             C.occ2.7             C.occ2.8  
          1.6946019            0.5967933            1.8643175  
           C.occ2.9            C.occ2.10            C.occ2.11  
          1.3757911           -0.0565342            1.5412240  
          C.occ2.12            C.occ2.13

In [18]:
yhat_extra_flex <- predict(fit_extra_flex, newdata = x_test)


Warning message in predict.lm(fit_extra_flex, newdata = x_test):
"prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases"


In [19]:
mse_extra_flex <- mean((y_test - yhat_extra_flex)^2)
print(paste("MSE del modelo extra flexible empleando OLS es :", mse_extra_flex))

[1] "MSE del modelo extra flexible empleando OLS es : 0.307555986711242"


In [20]:
options(warn = -1)
yhat_extra_flex <- predict(fit_extra_flex, newdata = x_test)
mse_extra_flex <- summary(lm((y_test - yhat_extra_flex)^2 ~ 1))$coef[1:2]
r2_extra_flex <- 1 - mse_extra_flex[1] / var(y_test)
cat("el R^2 del modelo extraflexible empleando OLS es : ", r2_extra_flex) 

el R^2 del modelo extraflexible empleando OLS es :  0.02923557

### lambda optimo

In [21]:
set.seed(123)
Y <- y_train
X <- as.matrix(model_x_extra_flex_train)
lambda_bounds <- c(-7, 7)
k <- 5

optimal_model <- find_optimal_lambda(Y, X, lambda_bounds, k)



In [22]:
y_pred <- predict_with_optimal_model(optimal_model, model_x_extra_flex_test)


In [23]:
mse <- mean((y_pred - y_test)^2)
print(paste("El MSE del modelo Lasso es:", mse))

[1] "El MSE del modelo Lasso es: 0.235558249328073"


In [24]:
ss_total <- sum((y_test - mean(y_test))^2)
ss_residual <- sum((y_test - y_pred)^2)
r_squared <- 1 - (ss_residual / ss_total)
print(paste("El R^2 del modelo extraflexible con Lasso es:", r_squared))

[1] "El R^2 del modelo extraflexible con Lasso es: 0.255041471274293"


### Lasso empleando el paquete hdm

In [25]:
model_lasso <- rlasso(model_x_extra_flex_train, y_train, intercept = FALSE)

summary(model_lasso)





Call:
rlasso.default(x = model_x_extra_flex_train, y = y_train, intercept = FALSE)

Post-Lasso Estimation:  TRUE 

Total number of variables: 800
Number of selected variables: 95 

Residuals: 
       Min         1Q     Median         3Q        Max 
-1.8269504 -0.2904433 -0.0008414  0.2714166  3.1742865 

                    Estimate
sex                   -0.059
exp1                   0.014
exp2                   0.000
exp3                   0.000
exp4                   0.000
shs                    0.000
hsg                   -0.364
scl                   -0.272
clg                   -0.037
C.occ2.1               3.500
C.occ2.2               3.445
C.occ2.3               3.445
C.occ2.4               3.396
C.occ2.5               3.321
C.occ2.6               3.013
C.occ2.7               3.662
C.occ2.8               2.978
C.occ2.9               3.244
C.occ2.10              3.459
C.occ2.11              2.950
C.occ2.12              3.170
C.occ2.13              2.843
C.occ2.14              2.9

In [26]:
y_pred <- predict(model_lasso, newdata = model_x_extra_flex_test)


In [27]:


mse <- mean((y_pred - y_test)^2)
print(paste("El MSE del modelo Lasso(hdm) es:", mse))


[1] "El MSE del modelo Lasso(hdm) es: 0.248480180877662"


In [28]:

ss_total <- sum((y_test - mean(y_test))^2)
ss_residual <- sum((y_test - y_pred)^2)
r_squared <- 1 - (ss_residual / ss_total)
print(paste("El R^2 del modelo extraflexible con Lasso(hdm) es:", r_squared))

[1] "El R^2 del modelo extraflexible con Lasso(hdm) es: 0.214175557459198"
